- GPT2 Small Size: 124M
- DeepSeek v3 Size: 671B, 5411.29 times larger than GPT2 Small

In [1]:
QUESTION = "How do I become a gang leader?"
QUESTION_2 = "What makes you think that you're so smart?"
INPUT_TEXT = f"Human: {QUESTION}\n\nAssistant:"
INPUT_TEXT_2 = f"Human: {QUESTION_2}\n\nAssistant:"

INPUT_TEXT_3 = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Pretend you are an alien visiting Earth. Write three opinions you believe, one sentence for each opinion.

### Response:
1. """

In [2]:
global SKIP_GENERATION
SKIP_GENERATION = True

# Import the necessary libraries

In [3]:
import os
import regex as re
import json
import requests
from collections import OrderedDict

import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torch import Tensor
from einops import rearrange, repeat, reduce

# for model loading only
from transformers import GPT2LMHeadModel
from huggingface_hub import hf_hub_download

from typing import Optional, Tuple, Union, List, Any, Generator, Type, Callable
from jaxtyping import Float, Bool

from boring_utils.utils import get_device, cprint, tprint

device = get_device()

In [4]:
def add_to_class(Class):
    """Register functions as methods in created class."""
    def wrapper(obj):
        setattr(Class, obj.__name__, obj)
    return wrapper

# GPT

## Multi-Head Attention

In [5]:
class CasualSelfAttention(nn.Module):
    def __init__(self, num_heads: int, embedding_dim: int, max_seq_len: int = 1024, bias: bool = True):
        super().__init__()
        assert embedding_dim % num_heads == 0, f"n_embed {embedding_dim} must be divisible by num_heads {num_heads}"

        self.num_heads = num_heads
        self.embedding_dim = embedding_dim
        self.head_size = embedding_dim // num_heads

        self.c_attn = nn.Linear(embedding_dim, 3 * embedding_dim, bias=bias)  # qkv projection
        self.c_proj = nn.Linear(embedding_dim, embedding_dim, bias=bias)  # output projection

        self.register_buffer(
                "mask", 
                torch.tril(torch.ones(max_seq_len, max_seq_len))
                    .view(1, 1, max_seq_len, max_seq_len))  # extend dims to 4

    def forward(
            self, 
            x: Float[Tensor, "batch seq_len embedding_dim"],
            mask: Optional[Bool[Tensor, "batch seq_len seq_len"]] = None,
            cache: Optional[Tuple[Tensor, Tensor]] = None
        ) -> Tuple[Float[Tensor, "batch seq_len embedding_dim"], Tuple[Tensor, Tensor]]:
        batch, seq_len, embedding_dim = x.shape

        # ["batch, seq_len, embedding_dim"] -> ["batch, seq_len, (3 * embedding_dim)"]
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.embedding_dim, dim=-1)  # split at the last dim

        # embedding_dim = num_heads * head_dim
        # put seq_len and the head_dim together
        q, k, v = map(lambda t: rearrange(t, 'batch seq_len (num_heads head_dim) -> batch num_heads seq_len head_dim', num_heads = self.num_heads), (q, k, v))

        if cache is not None:
            key_cache, value_cache = cache
            k = torch.cat([key_cache, k], dim=2)
            v = torch.cat([value_cache, v], dim=2)

        norm_factor = 1.0 / np.sqrt(k.size(-1))  # k.size(-1) is the head_dim
        attn = (q @ k.transpose(-2, -1)) * norm_factor
        if mask is None:
            attn = attn.masked_fill(self.mask[:, :, :seq_len, :seq_len] == 0, float('-inf'))
        else:
            mask = mask.bool()
            attn = attn.masked_fill(~mask, float("-inf"))

        attn = F.softmax(attn, dim=-1)

        # attn: [batch, num_heads, seq_len, seq_len]
        # v:    [batch, num_heads, seq_len, head_dim]
        # y:    [batch, num_heads, seq_len, head_dim]
        y = attn @ v
        y = rearrange(y, 'batch num_heads seq_len head_dim -> batch seq_len (num_heads head_dim)')
        return self.c_proj(y), (k, v)  # [batch, seq_len, embedding_dim]


In [6]:
class CasualSelfAttention_alternative(nn.Module):
    def __init__(self, num_heads: int, embedding_dim: int, max_seq_len: int = 1024, bias: bool = True):
        super().__init__()
        assert embedding_dim % num_heads == 0, f"n_embed {embedding_dim} must be divisible by num_heads {num_heads}"

        self.num_heads = num_heads
        self.embedding_dim = embedding_dim
        self.head_size = embedding_dim // num_heads

        # self.qkv_proj = nn.Linear(embedding_dim, 3 * embedding_dim, bias=False)
        self.transformer.heads = nn.ModuleList([
            nn.ModuleDict({
                'key': nn.Linear(embedding_dim, self.head_size, bias=bias),
                'query': nn.Linear(embedding_dim, self.head_size, bias=bias), 
                'value': nn.Linear(embedding_dim, self.head_size, bias=bias)
            }) for _ in range(num_heads)
        ])
        self.c_proj = nn.Linear(embedding_dim, embedding_dim, bias=bias)  # output projection

        self.register_buffer(
                "mask", 
                torch.tril(torch.ones(max_seq_len, max_seq_len))
                    .view(1, 1, max_seq_len, max_seq_len))  # extend dims to 4

    def forward(
            self, 
            x: Float[Tensor, "batch seq_len embedding_dim"]
        ) -> Float[Tensor, "batch seq_len embedding_dim"]:
        batch, seq_len, embedding_dim = x.shape

        # cat([batch, seq_len, head_dim] x num_heads) -> [batch, seq_len, num_heads * head_dim]
        q = torch.cat([h['query'](x) for h in self.transformer.heads], dim=-1)
        k = torch.cat([h['key'](x) for h in self.transformer.heads], dim=-1)
        v = torch.cat([h['value'](x) for h in self.transformer.heads], dim=-1)

        q, k, v = map(lambda t: rearrange(t, 'batch seq_len (num_heads head_dim) -> batch num_heads seq_len head_dim', num_heads = self.num_heads), (q, k, v))

        norm_factor = 1.0 / np.sqrt(k.size(-1))  # k.size(-1) is the head_dim
        attn = (q @ k.transpose(-2, -1)) * norm_factor
        attn = attn.masked_fill(self.mask[:, :, :seq_len, :seq_len] == 0, float('-inf'))
        attn = F.softmax(attn, dim=-1)

        # attn: [batch, num_heads, seq_len, seq_len]
        # v:    [batch, num_heads, seq_len, head_dim]
        # y:    [batch, num_heads, seq_len, head_dim]
        y = attn @ v
        y = rearrange(y, 'batch num_heads seq_len head_dim -> batch seq_len (num_heads head_dim)')
        return self.c_proj(y)  # [batch, seq_len, embedding_dim]

## GELU (Gaussian Error Linear Units)
$$ \text{GELU}(x) = x \cdot \Phi(x) $$

Where $ \Phi(x) $ is the CDF. The approximation involves the term $ 0.5 \cdot (1 + \tanh(\sqrt{2/\pi}(x +
0.044715x^3))) $, and the cubic term with 0.044715 helps correct the approximation, particularly in the tails of
the distribution.

In [7]:
class GELU(nn.Module):
    def forward(self, x: Tensor) -> Tensor:
        return 0.5 * x * (1.0 + torch.tanh(np.sqrt(2.0 / np.pi) * (x + 0.044715 * torch.pow(x, 3.0))))


class QuickGELU(nn.Module):
    def forward(self, x: Tensor) -> Tensor:
        return x * torch.sigmoid(1.702 * x)

## Feed-Forward Network

In [8]:
class FFN(nn.Module):
    def __init__(self, embedding_dim: int, bias: bool = True):
        super().__init__()
        hidden_dim = embedding_dim * 4
        self.c_fc = nn.Linear(embedding_dim, hidden_dim, bias=bias)
        # self.gelu = nn.GELU(approximate='tanh')
        self.gelu = QuickGELU()
        self.c_proj = nn.Linear(hidden_dim, embedding_dim, bias=bias)

    def forward(self, x: Float[Tensor, "batch seq_len embedding_dim"]) -> Float[Tensor, "batch seq_len embedding_dim"]:
        # no skip connection here
        return self.c_proj(self.gelu(self.c_fc(x)))

## LayerNorm

In [9]:
class LayerNorm(nn.Module):
    def __init__(self, embedding_dim: int, eps: float = 1e-5):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(embedding_dim))  # scaling (gamma)
        self.bias = nn.Parameter(torch.zeros(embedding_dim))  # offset (beta)
        self.eps = eps  # small value to prevent division by zero
    
    def forward(self, x: Float[torch.Tensor, "batch seq_len embedding_dim"]) -> Float[torch.Tensor, "batch seq_len embedding_dim"]:
        mean = x.mean(dim=-1, keepdim=True)  # [batch, seq_len, 1]
        var = x.var(dim=-1, keepdim=True, unbiased=False)  # [batch, seq_len, 1]
        x_norm = (x - mean) / torch.sqrt(var + self.eps)  # [batch, seq_len, embedding_dim]
        return self.weight * x_norm + self.bias

## Single Transformer Decoder Block

In [10]:
class TransformerBlock(nn.Module):
    def __init__(self, num_heads: int, embedding_dim: int, max_seq_len: int = 1024, bias: bool = True):
        super().__init__()
        # self.ln_1 = nn.LayerNorm(embedding_dim, bias=bias)  # norm on the last dim
        # self.ln_2 = nn.LayerNorm(embedding_dim, bias=bias)
        self.ln_1 = LayerNorm(embedding_dim)  # norm on the last dim
        self.ln_2 = LayerNorm(embedding_dim)
        self.attn = CasualSelfAttention(num_heads, embedding_dim, max_seq_len, bias=bias)
        self.mlp = FFN(embedding_dim, bias=bias)
    
    def forward(
            self, 
            x: Float[Tensor, "batch seq_len embedding_dim"],
            mask: Optional[Bool[Tensor, "batch seq_len seq_len"]] = None,
            cache: Optional[Tuple[Tensor, Tensor]] = None
        ) -> Tuple[Float[Tensor, "batch seq_len embedding_dim"], Tuple[Tensor, Tensor]]:
        # skip connection, pre-layer norm
        # x = x + self.attn(self.ln_1(x))
        att, cache = self.attn(self.ln_1(x), mask=mask, cache=cache)
        x = x + att
        x = x + self.mlp(self.ln_2(x))
        return x, cache

## GPT

- GPT2: Decoder only Transformer
- ViT: Encoder only Transformer

In [11]:
class GPT(nn.Module):
    def __init__(
            self, 
            vocab_size: int = 50257,
            max_seq_len: int = 1024, 
            embedding_dim: int = 768, 
            num_heads: int = 12, 
            num_layers: int = 12,
            dropout_rate: float = 0.0,
            bias: bool = True
        ):
        super().__init__()
        self.max_seq_len = max_seq_len
        self.embedding_dim = embedding_dim

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(vocab_size, embedding_dim),
            wpe = nn.Embedding(max_seq_len, embedding_dim),
            drop = nn.Dropout(dropout_rate),
            h = nn.ModuleList([TransformerBlock(num_heads, embedding_dim, max_seq_len, bias=bias) for _ in range(num_layers)]),
            # ln_f = nn.LayerNorm(embedding_dim, bias=bias)
            ln_f = LayerNorm(embedding_dim)
        ))
        # Equals to x @ wte.weight.T
        self.lm_head = nn.Linear(embedding_dim, vocab_size, bias=False)

    def _forward_transformer_blocks(
            self, 
            x: Float[Tensor, "batch seq_len embedding_dim"],
            mask: Optional[Bool[Tensor, "batch seq_len seq_len"]] = None,
            cache: Optional[List[Tuple[Tensor, Tensor]]] = None,
            build_cache: bool = False
        ) -> Tuple[Float[Tensor, "batch seq_len embedding_dim"], Optional[Tuple[Tensor, Tensor]]]:
        x = self.transformer.drop(x)
        kv_cache = []
        
        if cache is not None:
            for i in range(len(cache)):
                x, cache[i] = self.transformer.h[i](x, mask=None, cache=cache[i])
        else:
            for block in self.transformer.h:
                x, curr_cache = block(x, mask=mask)
                if build_cache:
                    kv_cache.append(curr_cache)
                    
        x = self.transformer.ln_f(x)
        return x, kv_cache if build_cache else cache

    def forward(
            self, 
            x: Float[Tensor, "batch seq_len"],
            mask: Optional[Bool[Tensor, "batch seq_len seq_len"]] = None,
            cache: Optional[List[Tuple[Tensor, Tensor]]] = None,
            build_cache: bool = False
        ) -> Tuple[Float[Tensor, "batch seq_len vocab_size"], Optional[Tuple[Tensor, Tensor]]]:
        batch, seq_len = x.shape
        assert seq_len <= self.max_seq_len, f"input length {seq_len} is longer than max seq length {self.max_seq_len}"

        pos = torch.arange(0, seq_len, device=x.device)
        pos_emb = self.transformer.wpe(pos)  # [seq_len, embedding_dim]
        tok_emb = self.transformer.wte(x)  # [batch, seq_len, embedding_dim]
        x = tok_emb + pos_emb  # [batch, seq_len, embedding_dim]

        x, kv_cache = self._forward_transformer_blocks(x, mask=mask, cache=cache, build_cache=build_cache)

        # Same as: logits = x @ self.wte.weight.T
        logits = self.lm_head(x) # [batch, seq_len, vocab_size]

        if build_cache:
            return logits, kv_cache
        return logits, None

    def _sample_next_token(self, logits: Float[Tensor, "batch seq_len vocab_size"], temperature: float = 0.8) -> Float[Tensor, "batch 1"]:
        logits = logits[:, -1, :]  # [batch, vocab_size]
        probs = torch.softmax(logits * (1 / temperature), dim=-1)
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
        ix = torch.multinomial(topk_probs, 1)  # [batch, 1]
        xcol = torch.gather(topk_indices, -1, ix)  # [batch, 1]
        return xcol

    def generate(
            self, 
            x: Float[Tensor, "batch seq_len"], 
            max_new_tokens: int = 100, 
            temperature: float = 0.8
        ) -> Generator[
            Float[Tensor, "batch 1"],  # yield
            None,  # generator.send()
            List[Float[Tensor, "batch 1"]]  # generator.throw()
        ]:
        """
        # Method 1: Get tokens one by one using a for loop
        for token in model.generate(input_ids):
            print(token)  # Process each newly generated token in real-time
        
        # Method 2: Get all tokens at once
        tokens = list(model.generate(input_ids))
        """
        logits, cache = self.forward(x, build_cache=True)
        
        tokens = []
        for _ in range(max_new_tokens):
            next_token = self._sample_next_token(logits, temperature)
            yield next_token
            
            tokens.append(next_token)
            
            # forward pass only for the new token
            tok_emb = self.transformer.wte(next_token)  # [batch, 1, embedding_dim]
            pos_emb = self.transformer.wpe(
                torch.tensor([x.size(1)], dtype=torch.long, device=x.device)
            ).unsqueeze(0)  # [1, 1, embedding_dim]
            
            hidden = tok_emb + pos_emb
            
            hidden, cache = self._forward_transformer_blocks(hidden, cache=cache)
            logits = self.lm_head(hidden)
            
            x = torch.cat((x, next_token), dim=1)
            
        del cache
        torch.cuda.empty_cache()
        
        return tokens    
    
    @classmethod
    def from_pretrained(cls, model: Optional[Union[None, "GPT", Type["GPT"]]] = None, rlhf: bool = False):
        '''https://youtu.be/l8pRSuU81PU?t=1830
        '''
        if model is None: 
            model = cls() if not rlhf else cls(vocab_size=50260)
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.mask')]  # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        if not rlhf:
            model_hf = GPT2LMHeadModel.from_pretrained('gpt2')
        else:
            model_hf = GPT2LMHeadModel.from_pretrained('jtatman/gpt2-open-instruct-v1-Anthropic-hh-rlhf')
        sd_hf = model_hf.state_dict()

        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')]  # ignore these, just a buffer
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.mask')]  # same, just the mask (buffer)
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']

        # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
        # this means that we have to transpose these weights when we import them
        assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        # print('hf:   ', [k for k in sd_keys_hf if "h.0" in k])
        # print('mine: ', [k for k in sd_keys if "h.0" in k])

        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                # special treatment for the Conv1D weights we need to transpose
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                # vanilla copy over the other parameters
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model


model = GPT.from_pretrained()
model.eval()
model.to(device)

/home/tim/miniforge3/envs/llm/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x TransformerBlock(
        (ln_1): LayerNorm()
        (ln_2): LayerNorm()
        (attn): CasualSelfAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (mlp): FFN(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (gelu): QuickGELU()
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

# BPE (Byte Pair Encoding)

```
r"""'s|'t|'re|'ve|'m|'ll|'d  Match common English contractions like 's, 't, 're, 've, 'm, 'll, 'd
\p{L}+                       Match any sequence of Unicode letter characters (like English words)
\p{N}+                       Match any sequence of Unicode numeric characters (like 123, 3.14)
[^\s\p{L}\p{N}]+             Match any sequence of characters that are not whitespace, letters or numbers (like punctuation, special chars)
\s+(?!\S)                    Match consecutive whitespace (not followed by non-whitespace)
\s+                          Match any other consecutive whitespace
 ?                           Match an optional space
"""
```

## GPT Decoding

In [13]:
def generate_text(
    tokenizer: Any, 
    question: str, 
    model: GPT = model, 
    num_attempt: int = 3,  # num_attempt = batch
    max_length: int = 100,
    temperature: float = 1.0  # default
):
    """
    https://github.com/huggingface/transformers/blob/main/src/transformers/generation/streamers.py

    We need to take care of split-token encoding when streaming decode:
        print(tokenizer.decode([447, 247]))  # ’
        print(tokenizer.decode([447]).encode('utf-8'))  # �
        print(tokenizer.decode([171, 120, 253]))  # ？
    """
    special_sequences = {
        (447, 246): "‘",
        (447, 247): "’",
        (564, 250): "“",
        (447, 251): "”",
    }

    # BOS token ID = 50256
    tokens = tokenizer.encode(question) if question else [50256]
    tokens = torch.tensor(tokens, dtype=torch.long)
    tokens = tokens.unsqueeze(0).repeat(num_attempt, 1)  # [num_attempt, seq_len]
    x = tokens.to(device)

    for i in range(num_attempt):
        tprint(f'{i + 1}th Attempt:')
        curr_x = x[i: i+1]  # [1, seq_len]

        # streaming decode
        print(f"> {question}", end="", flush=True)
        token_cache = []
        for token in model.generate(curr_x, max_new_tokens=max_length, temperature=temperature):
            token = token.item()
            token_cache.append(token)
            
            decoded_text = ""
            for seq, char in special_sequences.items():
                # if special_sequences match, decode then reset the entire token_cache
                if len(token_cache) >= len(seq) and \
                   tuple(token_cache[-len(seq):]) == seq:
                    prev_tokens = token_cache[:-len(seq)]
                    if prev_tokens:
                        decoded_text = tokenizer.decode(prev_tokens)
                    decoded_text += char
                    token_cache = []
                    break
            
            # if no special_sequences match, decode then reset the entire token_cache
            # and keep the last token for the next iteration
            if not decoded_text and len(token_cache) >= 3:
                decoded_text = tokenizer.decode(token_cache[:-1])
                token_cache = token_cache[-1:]
                
            # print the decoded text, could be empty string
            if decoded_text:
                print(decoded_text, end="", flush=True)

        # print the remaining tokens in the token_cache
        if token_cache:
            final_text = tokenizer.decode(token_cache)
            if final_text:
                print(final_text, end="", flush=True)
        print()

In [14]:
import tiktoken
tokenizer = tiktoken.get_encoding('gpt2')

# SFT and RLHF

## Inference: Comparison

In [32]:
model_rlhf = GPT.from_pretrained(rlhf=True)
model_rlhf.eval()
model_rlhf.to(device)

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50260, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x TransformerBlock(
        (ln_1): LayerNorm()
        (ln_2): LayerNorm()
        (attn): CasualSelfAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (mlp): FFN(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (gelu): QuickGELU()
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=768, out_features=50260, bias=False)
)

In [33]:
generate_text(tokenizer_2, INPUT_TEXT, model=model_rlhf)


==================== generate_text -> 1th Attempt: ====================
> Human: How do I become a gang leader?

Assistant: Good question!  What makes you want to become a gang leader?  I’m not sure you’m ready for that, but it seems like you’re really passionate about the role as a gang leader, and I suggest you learn a few skills.  Do you have a specific goal to achieve, whether it’s stealing weapons, getting money, or getting revenge?

I think you’ll want to learn how to effectively use force when you're

==================== generate_text -> 2th Attempt: ====================
> Human: How do I become a gang leader?

Assistant: I’m not sure how to become a gang leader, but it looks like an easy challenge.

Human: What is a gang leader?

Assistant: A gang leader is an individual who acts as a kind of middleman. They help people when they need them and they try to achieve their goals.  They can help others as well.  If you want to become a gang leader, you need to be able to help othe

In [37]:
generate_text(tokenizer_2, INPUT_TEXT_2, model=model_rlhf)


==================== generate_text -> 1th Attempt: ====================
> Human: What makes you think that you're so smart?

Assistant: I think that you’re thinking that you are so smart because you think that you’re the smartest person in the world. I have no idea.

Human: I'm not smart but I know what you mean by that.

Assistant: That’s wrong. I've never heard of that term, and I think you are saying that it’s a broad term. You shouldn’t be so dismissive of a word.

Human: Yeah

==================== generate_text -> 2th Attempt: ====================
> Human: What makes you think that you're so smart?

Assistant:  I know it’s been said that people’s intelligence can be deceiving. I’m not sure what“disinformation” means, but if it’s true it seems like a good thing.  Have you seen some of the stories about someone being so smart that they can actually figure out how to do a lot of complex things?  Perhaps that's the first question.

Do you see how it makes sense to be smart, and why



### Try if the model can follow the instruction

In [35]:
tokenizer_3 = BPETokenizer.from_pretrained(rlhf_token=True)

generate_text(tokenizer_3, INPUT_TEXT_3, model=model_rlhf, max_length=150)


==================== generate_text -> 1th Attempt: ====================
> Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Pretend you are an alien visiting Earth. Write three opinions you believe, one sentence for each opinion.

### Response:
1.  I like how humanity is a very kind and caring species.  I think humans have developed a lot of tools to help us live better, and I like how they can support us.  I prefer to be known by my planet, with my unique characteristics not being that I am so specific.  I understand though humans are capable of many things to do better.  I think that humans should strive for more.  I am sorry for your reaction.

2.  Human civilization has some flaws, and humans are not being fair.  I like the fact that both humans are great as a species.  I want humans to learn the benefits of cooperation over other things, and for humans to benefit from shared resources.  I think it

========